In [3]:
import pymysql
import prettytable as pt
import json
import pandas as pd
from tqdm import tqdm
import csv

In [27]:
db = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'mysql'
)
cursor = db.cursor()
cursor.execute("create database if not exists cs550")


cursor.execute("use cs550")

cursor.execute("select database()")

table = pt.from_db_cursor(cursor)
print(table)

db.close()

+------------+
| database() |
+------------+
|   cs550    |
+------------+


In [108]:
# simple analysis on review dataset
totalReviews = 157260921
columns = ['verified', 'image', 'style', 'asin', 'reviewerID', 'overall', 'reviewText', 'reviewTime', 'unixReviewTime', 'summary', 'reviewerName', 'vote']
column_dict = dict()
style = set()
dataPath = "../Data/All_Amazon_Review_5.json"
with open(dataPath, 'r', encoding="utf-8") as f:
    with tqdm(total=totalReviews, desc="Processing", leave=True, unit_scale=True) as pbar:
        index = 0
        line = f.readline()
        while line:
            pbar.update(1)
            row = json.loads(line)
            for key in row.keys():
                valueType = type(row[key])
                if key not in column_dict:
                    if valueType in [list, dict, str]:
                        column_dict[key] = (valueType.__name__, index, len(row[key]))
                    else:
                        column_dict[key] = (valueType.__name__)
                else:
                    if valueType in [list, dict, str] and len(row[key]) > column_dict[key][2]:
                        column_dict[key] = (valueType.__name__, index, len(row[key]))
            line = f.readline()
            index += 1
print(column_dict)

Processing: 100%|█████████▉| 157M/157M [30:48<00:00, 85.1kit/s]   

{'overall': 'float', 'verified': 'bool', 'reviewTime': ('str', 1, 11), 'reviewerID': ('str', 11586, 20), 'asin': ('str', 0, 10), 'reviewerName': ('str', 28782337, 1725), 'reviewText': ('str', 140467823, 35094), 'summary': ('str', 151619415, 1730), 'unixReviewTime': 'int', 'vote': ('str', 257028, 6), 'image': ('list', 136488683, 508), 'style': ('dict', 2377009, 7)}


In [20]:
# split dataset example
def splitColumns(selectedKeys, dataPath, totalRowNum, outputPath):
    with open(outputPath, "w", newline="") as outputFile:
        writer = csv.DictWriter(outputFile, selectedKeys)
        writer.writeheader()
        with open(dataPath, encoding="utf-8") as file:
            with tqdm(total=totalRowNum, desc="Processing", leave=True, unit_scale=True) as pbar:
                line = file.readline()
                while line:
                    pbar.update(1)
                    row = json.loads(line)
                    writer.writerow(dict([(key, row[key]) for key in selectedKeys]))
                    line = file.readline()

totalReviews = 157260921 # total number of reviews
dataPath = "../Data/All_Amazon_Review_5.json" # input dataset path
outputPath = "../Data/All_Amazon_Review_User_Item_Rating.csv" # output datset path
selectedKeys = ['reviewerID', 'asin', 'overall'] # choose columns to split from dataset
splitColumns(selectedKeys, dataPath, totalReviews, outputPath)

Processing: 100%|█████████▉| 157M/157M [27:03<00:00, 96.9kit/s]   


In [ ]:
totalMeta = 15023060
dataPath = "../Data/All_Amazon_Review_5.json"